# Initialization

## Imports

In [1]:
import sys
import platform

try:
    import tweedledum
    print("Tweedledum library is installed.")
except ImportError:
    if platform.system() == 'Darwin':
        print("Tweedledum library is not installed and your system (MacOS) is not compatible with this program.")
    else:
        print("Tweedledum library is not installed.")

Tweedledum library is not installed and your system (MacOS) is not compatible with this program.


In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import math
import pytest

from qiskit import IBMQ, Aer, assemble, transpile, QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.quantum_info.operators import Operator
from qiskit.providers.ibmq import least_busy
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import PhaseOracle
from qiskit.algorithms import Grover, AmplificationProblem

## Nonogram Code

In [9]:
# All possible nonogram descriptions for l=5 through l=5
# TODO: Replace with function to generate all bitstrings for a given clue

possible_d = {
    # l = 1
    "1/0;" : [0b0],
    "1/1;" : [0b1],
        
    # l = 2
    "2/0;" : [0b00],
    "2/1;" : [0b01,0b10],
    "2/2;" : [0b11],
    
    # l = 3
    "3/0;" : [0b000],
    "3/1;" : [0b100, 0b010,0b001],
    "3/2;" : [0b110,0b011],
    "3/3;" : [0b111],
    "3/1;1;" : [0b101],
    
    # l = 4
    "4/0;" : [0b0000],
    "4/1;" : [0b1000,0b0100, 0b0010,0b0001],
    "4/2;" : [0b1100,0b0110,0b0011],
    "4/3;" : [0b1110,0b0111],
    "4/4;" : [0b1111],
    "4/1;1;" : [0b1010,0b0101,0b1001],
    "4/2;1;" : [0b1101],
    "4/1;2;" : [0b1011],
              
    # l = 5
    "5/0;" : [0b00000],
    "5/1;" : [0b10000,0b01000,0b00100,0b00010,0b00001],
    "5/2;" : [0b11000,0b01100,0b00110, 0b00011],
    "5/3;" : [0b11100,0b01110,0b00111],
    "5/4;" : [0b11110,0b01111],
    "5/5;" : [0b11111],
    "5/1;1;" : [0b10100,0b10010,0b10001,0b01010,0b01001,0b00101],
    "5/1;2;" : [0b10011,0b10110,0b01011],
    "5/1;3;" : [0b10111],
    "5/2;1;" : [0b11001,0b11010,0b01101,],
    "5/2;2;" : [0b11011],
    "5/3;1;" : [0b11101],
    "5/1;1;1;" : [0b10101],
}

# Takes an n x d board and returns the variables for each row, column
def var_clauses(n, d=None):
    if d is None:
        d = n
    X = np.arange(n*d).reshape((n,d))
    col_vars = []
    row_vars = []
    for row in range(n):
        row_vars.append(list(X[row, :]))
    for col in range(d):
        col_vars.append(list(X[:, col]))
    return row_vars, col_vars

def display_nonogram(bit_string, n, d):
    if (n*d > len(bit_string)):
        raise Exception(f'bitstring is length {len(bit_string)}, expected {n * d}')
    
    puzzle_array = np.zeros((n ,d))
    for i in range(n):
        for j in range(d):
            puzzle_array[i,j] = int(bit_string[i*d+j])
    print('╔' + '═'*d + '╗')
    
    for i in range(n):
        print_row = '║'
        for j in range(d):
            if puzzle_array[i,j] == 0:
                print_row += '░'
            else:
                print_row += '▓'
        print_row += '║ ' 
        print(print_row)
    print('╚' + '═'*d + '╝')
    
def validate(rows, cols, r_clues, c_clues):
    if (len(r_clues) != rows):
        raise Exception(f"Error: Number of clues {len(r_clues)} invalid to row size {rows}")
    if (len(c_clues) != cols):
        raise Exception(f"Error: Number of clues {len(c_clues)} invalid to row size {cols}")
    # TODO: Flesh out clue validation with invalid clue for given length/width
    return True

## Quantum Oracle Construction

In [6]:
def boolean_phase_oracle(row_clues, col_clues, n, d = None, debug_mode = False):
    if d is None:
        d = n
    boolean_statement = ""
    r_v, c_v = var_clauses(n, d)
    
    for r_idx, r_clue in enumerate(row_clues):
        bit_strings = possible_d[f"{n}/{';'.join(map(str, r_clue))};"]
        clauses = []
        for b_idx, bitstring in enumerate(bit_strings):
            clause = ""
            for c_idx in range(d):
                if bitstring & (1 << c_idx):
                    clause += f'v{r_v[r_idx][c_idx]}&'
                else:
                    clause += f'~v{r_v[r_idx][c_idx]}&'
            clauses.append("(" + clause[:-1] + ")")
        boolean_statement += "(" + "|".join(clauses) + ")&"
        
 # iterate over column clues (same as with row clues, but with transposed variables)
    for c_idx, c_clue in enumerate(col_clues):
        bitstrings = possible_d[f"{d}/{';'.join(map(str, c_clue))};"]
        clauses = []
        for b_idx, bitstring in enumerate(bitstrings):
            clause = ""
            for r_idx in range(n):
                if bitstring & (1 << r_idx):
                    clause += f"v{r_v[r_idx][c_idx]}&"
                else:
                    clause += f"~v{r_v[r_idx][c_idx]}&"
            clauses.append("(" + clause[:-1] + ")")
        boolean_statement += "(" + "|".join(clauses) + ")&"
 
    # remove trailing "&"
    boolean_statement = boolean_statement[:-1]
    return PhaseOracle(boolean_statement) if not debug_mode else boolean_statement
 

# Driver Code

In [7]:
# Define nonogram [size and clues]
solutions = ["0110"] # Leave empty if unknown

if not solutions or len(solutions) == 0:
    num_solutions = 1 # Assumed
else:
    num_solutions = len(solutions) # Given

# Make sure all rows/cols have a clue
row_clues = [(1,),(0,),]
rows = len(row_clues)

col_clues = [(0,),(1,),]
columns = len(col_clues)

try:
    validate(rows, columns, row_clues, col_clues)
    
    num_iterations = math.ceil(math.pi/4 * math.sqrt(2**(rows*columns)/num_solutions))
    
    oracle = boolean_phase_oracle(row_clues=row_clues,col_clues=col_clues,n=rows,d=columns)
    problem = AmplificationProblem(oracle=oracle)
    algorithm = Grover(iterations=num_iterations)
    
    circuit = algorithm.construct_circuit(problem)
    circuit.measure_all()
    print(circuit)
    
except Exception as e: 
    print(e)

"The 'tweedledum' library is required to use 'PhaseOracle'. You can install it with 'pip install tweedledum'."


## Results

In [8]:
print("Expected: ")
for solution in solutions:
    print(solution)
    display_nonogram(solution, rows, columns)

backend = Aer.get_backend('aer_simulator')  
job = execute(circuit, backend, shots=1024)
result = job.result()
counts = result.get_counts(circuit)
# plot_histogram(counts)

sorted_counts = dict(sorted(counts.items(), key= lambda item: item[1], reverse = True))
top_three = dict(list(sorted_counts.items())[:3])
print("Actual: ")
print(top_three[0])
display_nonogram(top_three[0], rows, columns)
plot_histogram(top_three)

Expected: 
0110
╔══╗
║░▓║ 
║▓░║ 
╚══╝


NameError: name 'circuit' is not defined